###  This is a template for the get_date monte carlo function.  

In [1]:
import random
import rasterio as rio
import numpy as np
from numpy.random import choice
from datetime import datetime, date, timedelta


In [2]:
def get_date(start_date, end_date, delta_time):
    # initialize date list with start_date and increment by delta_time
    dates = start_date
    date_array = [start_date]
    while dates < end_date:
        date_array.append(dates)
        dates += delta_time

    # randomly select time based on delta_time increment
    return random.choice(date_array) 

In [3]:

start_date = datetime(2015, 1, 1, 0, 30)
end_date   = datetime(2018, 12, 31, 23, 55)
delta_time = timedelta(minutes = 30)
spill_date = get_date(start_date, end_date, delta_time)

In [4]:
spill_date

datetime.datetime(2015, 3, 10, 17, 30)

In [5]:
spill_date.month

3

In [6]:
spill_date.hour

17

###  The code below develops the process of selecting the date by first choosing a month that is weighed by Vessel Traffic Exposure

##### Get month

In [17]:
from array import *
import rasterio as rio

# specify where geotiff files are located
geotiff_directory = '/Users/rmueller/Data/MIDOSS/AIS/ShipTrackDensityGeoTIFFs/'
n_locations = 1

# Load GeoTIFF files for each month and add up VTE
months = array('i',range(1,13))
total_vte_by_month = []

for month in months:
    
    # The filenames are formated as "all_2018_MM.tif"
    f_name = f'{geotiff_directory}all_2018_{month:02.0f}.tif'
    
    # open GeoTIFF file for reading
    traffic_reader = rio.open(f_name)
    
    # load data in a way that automatically closes file when finished
    with traffic_reader as dataset:

        # resample data to target shape
        data = dataset.read(1)

        # remove no-data values and singular dimension 
        data = np.squeeze(data)
        data[data < 0] = 0

        total_vte_by_month.append(data.sum())    

    print(f_name)
                
print(total_vte_by_month)                            

/Users/rmueller/Data/MIDOSS/AIS/ShipTrackDensityGeoTIFFs/all_2018_01.tif
/Users/rmueller/Data/MIDOSS/AIS/ShipTrackDensityGeoTIFFs/all_2018_02.tif
/Users/rmueller/Data/MIDOSS/AIS/ShipTrackDensityGeoTIFFs/all_2018_03.tif
/Users/rmueller/Data/MIDOSS/AIS/ShipTrackDensityGeoTIFFs/all_2018_04.tif
/Users/rmueller/Data/MIDOSS/AIS/ShipTrackDensityGeoTIFFs/all_2018_05.tif
/Users/rmueller/Data/MIDOSS/AIS/ShipTrackDensityGeoTIFFs/all_2018_06.tif
/Users/rmueller/Data/MIDOSS/AIS/ShipTrackDensityGeoTIFFs/all_2018_07.tif
/Users/rmueller/Data/MIDOSS/AIS/ShipTrackDensityGeoTIFFs/all_2018_08.tif
/Users/rmueller/Data/MIDOSS/AIS/ShipTrackDensityGeoTIFFs/all_2018_09.tif
/Users/rmueller/Data/MIDOSS/AIS/ShipTrackDensityGeoTIFFs/all_2018_10.tif
/Users/rmueller/Data/MIDOSS/AIS/ShipTrackDensityGeoTIFFs/all_2018_11.tif
/Users/rmueller/Data/MIDOSS/AIS/ShipTrackDensityGeoTIFFs/all_2018_12.tif
[40241.77, 39879.977, 48065.723, 46474.059, 57431.367, 57617.504, 58803.598, 60949.133, 57478.613, 55143.496, 51053.902, 472

In [18]:
vte_probability = total_vte_by_month / np.sum(total_vte_by_month)

In [19]:
vte_probability

array([ 0.06487016,  0.06428695,  0.07748246,  0.07491668,  0.09257998,
        0.09288004,  0.09479203,  0.09825066,  0.09265614,  0.08889191,
        0.08229944,  0.07609361], dtype=float32)

In [20]:
np.sum(vte_probability)

1.0

#### Randomly select month based on weighting by vessel traffic

In [21]:
month_random = choice( months, 
                       n_locations , 
                       p = vte_probability
                     )

#### Now that month is selected, we need to choose day, year, and time.  We weight these all the same

In [63]:
time_period = end_date - start_date
time_period_inhours = np.int(time_period.total_seconds()/3600)
date_arr = [start_date + timedelta(hours=i) for i in range(0,time_period_inhours+1)]

In [64]:
date_arr[-1]

datetime.datetime(2018, 12, 31, 23, 30)

In [68]:
date_arr_select = []
for days in date_arr:
    if days.month == month_random:
        date_arr_select.append(days)

In [71]:
random.choice(date_arr_select)

datetime.datetime(2017, 11, 8, 5, 30)